In [ ]:
# !pip install geopandas earthengine-api rasterio pillow

import ee
import geopandas as gpd
import os
import time


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

# @title Setup
ee.Authenticate()
ee.Initialize(project='nokia-project-runway-detect')

GEOJSON_PATH = '/content/drive/MyDrive/Illegal-Airstrips-NYT-Intercept-Public.geojson'
EXPORT_DIR = '/content/drive/MyDrive/GEE_Exports'
PIXELS = 512
RESOLUTION = 10
BUFFER_METERS = (PIXELS * RESOLUTION) / 2
YEAR = 2023


In [ ]:

# @title modules

def create_square_box(point_geom):
    """Create 5.12km square (2560m × 2560m) around a point"""
    point = ee.Geometry.Point([point_geom.x, point_geom.y])
    return point.buffer(BUFFER_METERS).bounds()

def get_composite_image(geom):
    """Return 9-band image clipped to geometry"""
    s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(geom) \
        .filterDate(f'{YEAR}-01-01', f'{YEAR}-12-31') \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'])

    s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(geom) \
        .filterDate(f'{YEAR}-01-01', f'{YEAR}-12-31') \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .select(['VV', 'VH'])

    s2_img = s2.median()
    s1_img = s1.median()
    ratio = s1_img.select('VV').divide(s1_img.select('VH')).rename('VV_VH_ratio')

    return s2_img.addBands(s1_img).addBands(ratio).clip(geom)

def export_image(geom, airstrip_id):
    image = get_composite_image(geom)
    region = geom.bounds().getInfo()['coordinates']
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'id_{airstrip_id}',
        folder=EXPORT_DIR,
        fileNamePrefix=f'id_{airstrip_id}',
        region=region,
        scale=RESOLUTION,
        crs='EPSG:4326',
        maxPixels=1e9,
        fileFormat='GeoTIFF'
    )
    task.start()
    print(f'📤 Export started: {airstrip_id}')




In [ ]:
# @title  Main

gdf = gpd.read_file(GEOJSON_PATH)
print(f"Total airstrips: {len(gdf)}")

for i, row in gdf.iloc[1042:].iterrows():
    point = row.geometry
    airstrip_id = row['Airstrip ID']
    try:
        geom = create_square_box(point)
        export_image(geom, airstrip_id)
        time.sleep(3)  # GEE rate limit friendly
    except Exception as e:
        print(f"❌ Failed {airstrip_id}: {e}")

In [ ]:
# @title PNG conversion

from PIL import Image
import numpy as np
import rasterio

def convert_geotiffs_to_pngs(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for fname in os.listdir(input_folder):
        if fname.endswith('.tif'):
            tiff_path = os.path.join(input_folder, fname)
            png_path = os.path.join(output_folder, fname.replace('.tif', '.png'))
            with rasterio.open(tiff_path) as src:
                # Sentinel-2 bands are: B2 (Blue), B3 (Green), B4 (Red)
                b2 = src.read(1)  # Blue
                b3 = src.read(2)  # Green
                b4 = src.read(3)  # Red

                rgb = np.stack([b4, b3, b2], axis=-1)  # Rearrange to RGB
                rgb = np.nan_to_num(rgb)

                # Stretch to 0-255 safely
                rgb = (255 * (rgb - rgb.min()) / (rgb.max() - rgb.min())).astype(np.uint8)
                img = Image.fromarray(rgb, mode='RGB')
                img.save(png_path)
                print(f'🖼️ Saved PNG: {png_path}')

# After all exports are done and appear in Drive:
convert_geotiffs_to_pngs(EXPORT_DIR, '/content/drive/MyDrive/png_chips')


In [ ]:
#@title  Data Check

import os
import rasterio

EXPORT_DIR = '/content/drive/MyDrive/GEE_Exports'

# Define expected band roles
band_roles = [
    ('B2',  'Blue'),
    ('B3',  'Green'),
    ('B4',  'Red'),
    ('B8',  'NIR'),
    ('B11', 'SWIR1'),
    ('B12', 'SWIR2'),
    ('VV',  'Radar VV'),
    ('VH',  'Radar VH'),
    ('VV/VH_ratio', 'Radar Ratio')
]

tif_files = [f for f in os.listdir(EXPORT_DIR) if f.endswith('.tif')]

for fname in tif_files[:3]:
    path = os.path.join(EXPORT_DIR, fname)
    print(f"\n📂 File: {fname}")

    try:
        with rasterio.open(path) as src:
            print(f"  → Dimensions: {src.width} x {src.height}")
            print(f"  → Total Bands: {src.count}")

            for i in range(src.count):
                band_name, meaning = band_roles[i] if i < len(band_roles) else ('Unknown', 'Unknown')
                stats = src.read(i + 1)
                print(f"    • Band {i+1}: {band_name} — {meaning}")
                print(f"      Range: min={stats.min():.2f}, max={stats.max():.2f}, dtype={stats.dtype}")
    except Exception as e:
        print(f"  ❌ Could not read {fname}: {e}")


In [ ]:
#@title  Check export status

import ee
from datetime import datetime

def check_gee_export_status(limit):
    """Prints status of the most recent Earth Engine export tasks"""
    tasks = ee.batch.Task.list()[:limit]
    print(f"\n📋 Checking last {limit} Earth Engine export tasks:")

    for task in tasks:
        desc = task.config.get('description', 'No description')
        status = task.status()
        state = status['state']

        print(f"• {desc} → {state}")

        if state == 'FAILED':
            print(f"  ❌ Error: {status.get('error_message', 'No details')}")
        elif state == 'COMPLETED':
            print(f"  ✅ Done. Created at: {datetime.fromtimestamp(status['creation_timestamp_ms']/1000)}")
        elif state in ['READY', 'RUNNING']:
            print(f"  ⏳ Still processing...")

# Run it
check_gee_export_status(50)  # Increase limit if needed
